In [11]:
from datetime import datetime
from datetime import timedelta
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [12]:
url = "https://datasets.imdbws.com/name.basics.tsv.gz"
df1 = pd.read_csv(url, sep = '\t')

In [13]:
df1.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0043044"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"


In [14]:
df1.shape

(14013602, 6)

In [15]:
#afficher le nombre de valeurs manquantes par colonne
df1.isna().sum()


nconst               0
primaryName          9
birthYear            0
deathYear            0
primaryProfession    0
knownForTitles       0
dtype: int64

In [16]:
#afficher le nombre de valeurs \N  par colonne
(df1 == '\\N').sum()


nconst                      0
primaryName                50
birthYear            13380365
deathYear            13776109
primaryProfession     2720840
knownForTitles        1586468
dtype: int64

In [17]:
df1.replace('\\N', np.nan, inplace=True)


In [18]:
#remplacer les valeurs \N par des NaN dans tout le DataFrame
df1.isna().sum()


nconst                      0
primaryName                59
birthYear            13380365
deathYear            13776109
primaryProfession     2720840
knownForTitles        1586468
dtype: int64

In [19]:
# nous avons 59 primaryname qui manquent, on va les supprimer, sans nom on ne peut rien faire
df1 = df1[df1['primaryName'].notna()]
df1.shape


(14013543, 6)

In [20]:
#convertir les colonnes dates qui sont en objet au type float
df1['birthYear'] = pd.to_numeric(df1['birthYear'], errors='coerce')
df1['deathYear'] = pd.to_numeric(df1['deathYear'], errors='coerce')


In [21]:
df1[['birthYear', 'deathYear']].dtypes

birthYear    float64
deathYear    float64
dtype: object

In [22]:
# Vérifier les doublons dans la colonne 'primaryName'
doublons = df1[df1.duplicated(subset=['primaryName'], keep=False)]




In [23]:
# Supprimer les doublons en gardant la première occurrence
df1 = df1.drop_duplicates(subset=['primaryName'], keep='first')


In [24]:
# Créer un nouveau DataFrame sans les lignes où 'primaryProfession' est NaN
df1_cleaned = df1.dropna(subset=['primaryProfession'])



In [25]:
df1_cleaned = df1_cleaned.dropna(subset=['primaryName'])


In [26]:
# Séparer les professions dans chaque ligne et extraire les valeurs uniques
professions = df1_cleaned['primaryProfession'].dropna().str.split(',').explode().unique()
print(professions)


['actor' 'miscellaneous' 'producer' 'actress' 'soundtrack'
 'archive_footage' 'music_department' 'writer' 'director' 'stunts'
 'make_up_department' 'composer' 'assistant_director' 'camera_department'
 'music_artist' 'art_department' 'editor' 'cinematographer'
 'casting_director' 'executive' 'visual_effects' 'costume_designer'
 'script_department' 'editorial_department' 'costume_department'
 'animation_department' 'talent_agent' 'archive_sound'
 'production_designer' 'special_effects' 'production_manager'
 'art_director' 'sound_department' 'casting_department'
 'location_management' 'set_decorator' 'transportation_department'
 'choreographer' 'legal' 'manager' 'podcaster' 'publicist' 'assistant'
 'production_department' 'accountant' 'electrical_department']


In [27]:
# Tri des professions
sorted_professions = sorted(professions)
print(sorted_professions)


['accountant', 'actor', 'actress', 'animation_department', 'archive_footage', 'archive_sound', 'art_department', 'art_director', 'assistant', 'assistant_director', 'camera_department', 'casting_department', 'casting_director', 'choreographer', 'cinematographer', 'composer', 'costume_department', 'costume_designer', 'director', 'editor', 'editorial_department', 'electrical_department', 'executive', 'legal', 'location_management', 'make_up_department', 'manager', 'miscellaneous', 'music_artist', 'music_department', 'podcaster', 'producer', 'production_department', 'production_designer', 'production_manager', 'publicist', 'script_department', 'set_decorator', 'sound_department', 'soundtrack', 'special_effects', 'stunts', 'talent_agent', 'transportation_department', 'visual_effects', 'writer']


In [28]:
profession_counts = df1_cleaned['primaryProfession'].dropna().str.split(',').explode().value_counts()
print(profession_counts)


primaryProfession
actor                        2498014
actress                      1654457
miscellaneous                1110433
producer                      945714
writer                        739241
director                      594697
camera_department             592474
art_department                384418
cinematographer               299080
sound_department              292132
composer                      276814
editor                        276677
music_department              229983
assistant_director            212115
visual_effects                191017
make_up_department            190233
animation_department          181255
production_manager            175770
archive_footage               161448
editorial_department          146923
costume_department            135818
soundtrack                    126723
transportation_department      81051
art_director                   77237
script_department              73371
stunts                         72284
location_management 

In [29]:
# Filtrer pour garder uniquement les "actor", "actress" et "director"
df1_cleaned = df1_cleaned[df1_cleaned['primaryProfession'].str.contains('actor|actress|director', na=False)]



In [31]:
print(df1_cleaned.head())
print(df1_cleaned.shape)  # Nombre de lignes et colonnes après le filtrage


      nconst      primaryName  birthYear  deathYear  \
0  nm0000001     Fred Astaire     1899.0     1987.0   
1  nm0000002    Lauren Bacall     1924.0     2014.0   
2  nm0000003  Brigitte Bardot     1934.0        NaN   
3  nm0000004     John Belushi     1949.0     1982.0   
4  nm0000005   Ingmar Bergman     1918.0     2007.0   

                    primaryProfession                           knownForTitles  
0        actor,miscellaneous,producer  tt0072308,tt0050419,tt0053137,tt0043044  
1  actress,soundtrack,archive_footage  tt0037382,tt0075213,tt0117057,tt0038355  
2   actress,music_department,producer  tt0057345,tt0049189,tt0056404,tt0054452  
3       actor,writer,music_department  tt0072562,tt0077975,tt0080455,tt0078723  
4               writer,director,actor  tt0050986,tt0069467,tt0050976,tt0083922  
(4832802, 6)


In [32]:
df1_cleaned.reset_index(drop=True, inplace=True)


In [34]:
# Filtrer les lignes selon les critères
df1_filtered = df1_cleaned[
    (df1_cleaned['birthYear'] > 1900) &
    ((df1_cleaned['deathYear'] > 1990) | (df1_cleaned['deathYear'].isna())) &
    (df1_cleaned['birthYear'].notna())
]

# Afficher les premières lignes pour vérification
print(df1_filtered.head())
print(f"Nombre de lignes après filtrage : {df1_filtered.shape[0]}")


       nconst      primaryName  birthYear  deathYear  \
1   nm0000002    Lauren Bacall     1924.0     2014.0   
2   nm0000003  Brigitte Bardot     1934.0        NaN   
4   nm0000005   Ingmar Bergman     1918.0     2007.0   
7   nm0000008    Marlon Brando     1924.0     2004.0   
12  nm0000013        Doris Day     1922.0     2019.0   

                     primaryProfession  \
1   actress,soundtrack,archive_footage   
2    actress,music_department,producer   
4                writer,director,actor   
7                actor,director,writer   
12      actress,producer,miscellaneous   

                             knownForTitles  
1   tt0037382,tt0075213,tt0117057,tt0038355  
2   tt0057345,tt0049189,tt0056404,tt0054452  
4   tt0050986,tt0069467,tt0050976,tt0083922  
7   tt0078788,tt0068646,tt0047296,tt0070849  
12  tt0048317,tt0045591,tt0053172,tt0055100  
Nombre de lignes après filtrage : 337404


In [35]:
df1_filtered.head(30)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0069467,tt0050976,tt0083922"
7,nm0000008,Marlon Brando,1924.0,2004.0,"actor,director,writer","tt0078788,tt0068646,tt0047296,tt0070849"
12,nm0000013,Doris Day,1922.0,2019.0,"actress,producer,miscellaneous","tt0048317,tt0045591,tt0053172,tt0055100"
13,nm0000014,Olivia de Havilland,1916.0,2020.0,"actress,soundtrack,archive_footage","tt0031381,tt0041452,tt0029843,tt0040806"
15,nm0000016,Georges Delerue,1925.0,1992.0,"composer,music_department,actor","tt0091763,tt0096320,tt0069946,tt0080610"
16,nm0000017,Marlene Dietrich,1901.0,1992.0,"actress,music_department,soundtrack","tt0051201,tt0055031,tt0052311,tt0021156"
17,nm0000018,Kirk Douglas,1916.0,2020.0,"actor,producer,director","tt0080736,tt0054331,tt0049456,tt0050825"
18,nm0000019,Federico Fellini,1920.0,1993.0,"writer,director,actor","tt0056801,tt0047528,tt0071129,tt0050783"


In [36]:
# Convertir 'birthYear' en datetime, en prenant l'année comme une date de début de l'année
df1['birthYear'] = pd.to_datetime(df1['birthYear'], format='%Y', errors='coerce')

# Convertir 'deathYear' en datetime, en prenant l'année comme une date de fin de l'année (si elle existe)
df1['deathYear'] = pd.to_datetime(df1['deathYear'], format='%Y', errors='coerce')

# Vérifier le résultat
print(df1.info())


<class 'pandas.core.frame.DataFrame'>
Index: 10757376 entries, 0 to 14013600
Data columns (total 6 columns):
 #   Column             Dtype         
---  ------             -----         
 0   nconst             object        
 1   primaryName        object        
 2   birthYear          datetime64[ns]
 3   deathYear          datetime64[ns]
 4   primaryProfession  object        
 5   knownForTitles     object        
dtypes: datetime64[ns](2), object(4)
memory usage: 574.5+ MB
None


In [37]:
df1 = df1_filtered
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
Index: 337404 entries, 1 to 4832791
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   nconst             337404 non-null  object 
 1   primaryName        337404 non-null  object 
 2   birthYear          337404 non-null  float64
 3   deathYear          82034 non-null   float64
 4   primaryProfession  337404 non-null  object 
 5   knownForTitles     333663 non-null  object 
dtypes: float64(2), object(4)
memory usage: 18.0+ MB
None


In [38]:
# Sauvegarder le DataFrame nettoyé 
df1_filtered.to_csv('C:/Users/Win10/Desktop/projet 2 bdd/name_basics_mourad_fin.csv', index=False)